# Implementing and Testing the Recommendation System with Python
## NoSQL Final Project
Walid El Otmani - Graph Database for a Brazilian E-Commerce Store

In [ ]:
# Create a connection to the Neo4j database
from neo4j import GraphDatabase
import pandas as pd

URI = "bolt://localhost:7687"
USER = "neo4j"
PASSWORD = "password"

driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))

def run_query(query, params=None):
    with driver.session() as session:
        result = session.run(query, params or {})
        return [r.data() for r in result]

def run_query_df(query, params=None):
    return pd.DataFrame(run_query(query, params))

print("Connected to Neo4j!")

Connected to Neo4j!


In [ ]:
# Creating a function to get recommendable products based on co-purchase patterns, so that products frequently bought together are recommended.
def get_recommendable_products(limit=100):
    query = """
    MATCH (p:Product)
    WHERE p.in_demo = true

    MATCH (p)<-[:CONTAINS_PRODUCT]-(oi:OrderItem)
          <-[:HAS_ITEM]-(o:Order)
    MATCH (o)-[:HAS_ITEM]->(otherOi:OrderItem)
          -[:CONTAINS_PRODUCT]->(other:Product)
    WHERE other.product_id <> p.product_id
      AND other.in_demo = true

    WITH p, COUNT(DISTINCT other) AS numRecommended
    WHERE numRecommended > 0
    RETURN 
      p.product_id AS id,
      p.display_name AS name,
      p.product_category_name AS category,
      p.demo_price AS price,
      numRecommended
    ORDER BY numRecommended DESC
    LIMIT $limit
    """
    return run_query_df(query, {"limit": limit})

In [ ]:
# Testing the recommendation function
recommendable = get_recommendable_products(50)
recommendable

,id,name,category,price,numRecommended
0,d5991653e037ccb7af6ed7d94246b249,External Hard Drive,informatica_acessorios,79.99,7
1,b7d94dc0640c7025dc8e3b46b52d8239,Webcam,informatica_acessorios,49.99,7
2,43b54d1fc56ff394092a3dff6be2d39f,Natural Lip Balm,beleza_saude,4.99,6
3,aa70ae9ee9a798d44ae6e00ce6d82cb1,Organic Face Mask,beleza_saude,12.99,6
4,0a4093a4af429dc0a9334300e5c13ae5,Organic Face Mask,beleza_saude,12.99,5
5,3f14d740544f37ece8a9e7bc8349797e,Mechanical Keyboard,informatica_acessorios,89.99,5
6,e0cf79767c5b016251fe139915c59a26,Essential Oils Collection,beleza_saude,25.00,5
7,4f0abfbee7569e43b80f9aaa256a9bf8,Natural Lip Balm,beleza_saude,4.99,5
8,1dc7685f4fdb9622d84ae2ec658d5bbf,Wireless Mouse,informatica_acessorios,24.99,5
9,909b87db6cb3a7ab26bd03cc59860136,Laptop Stand,informatica_acessorios,29.99,4


In [ ]:
# Creating a function to recommend products based on a given product_id using co-purchase patterns.
def recommend_products(product_id, limit=3):
    query = """
    MATCH (p:Product {product_id: $productId})
    WHERE p.in_demo = true
    
    MATCH (p)<-[:CONTAINS_PRODUCT]-(oi:OrderItem)
          <-[:HAS_ITEM]-(o:Order)
    MATCH (o)-[:HAS_ITEM]->(otherOi:OrderItem)
          -[:CONTAINS_PRODUCT]->(other:Product)
    WHERE other.product_id <> p.product_id
      AND other.in_demo = true

    WITH other, COUNT(DISTINCT o) AS coPurchaseCount
    RETURN 
      other.product_id AS id,
      other.display_name AS name,
      other.product_category_name AS category,
      other.demo_price AS price,
      coPurchaseCount
    ORDER BY coPurchaseCount DESC
    LIMIT $limit
    """
    return run_query_df(query, {"productId": product_id, "limit": limit})

In [ ]:
# Testing the recommandable products function
recommendable = get_recommendable_products(5)
recommendable

,id,name,category,price,numRecommended
0,b7d94dc0640c7025dc8e3b46b52d8239,Webcam,informatica_acessorios,49.99,7
1,d5991653e037ccb7af6ed7d94246b249,External Hard Drive,informatica_acessorios,79.99,7
2,aa70ae9ee9a798d44ae6e00ce6d82cb1,Organic Face Mask,beleza_saude,12.99,6
3,43b54d1fc56ff394092a3dff6be2d39f,Natural Lip Balm,beleza_saude,4.99,6
4,e0cf79767c5b016251fe139915c59a26,Essential Oils Collection,beleza_saude,25.00,5


In [ ]:
# Testing the recommend_products function with a sample product_id taken from the recommendable products table
recommend_products("d5991653e037ccb7af6ed7d94246b249", limit=3)

,id,name,category,price,coPurchaseCount
0,909b87db6cb3a7ab26bd03cc59860136,Laptop Stand,informatica_acessorios,29.99,1
1,36f60d45225e60c7da4558b070ce4b60,Laptop Stand,informatica_acessorios,29.99,1
2,bf3f9743aeca956fe97611364fec85d9,Webcam,informatica_acessorios,49.99,1


In [ ]:
# Another test with a different product_id
recommend_products("b7d94dc0640c7025dc8e3b46b52d8239", limit=5)

,id,name,category,price,coPurchaseCount
0,3ce943997ff85cad84ec6770b35d6bcd,Mechanical Keyboard,informatica_acessorios,89.99,5
1,6c90c0f6c2d89eb816b9e205b9d6a36a,External Hard Drive,informatica_acessorios,79.99,2
2,9616caad30917a469d87d2f95cd603cb,USB-C Hub,informatica_acessorios,34.99,1
3,55939df5d8d2b853fbc532bf8a00dc32,Wireless Mouse,informatica_acessorios,24.99,1
4,27e83703cdd809616f51c1d456314ed7,External Hard Drive,informatica_acessorios,79.99,1


In [ ]:
# Another test with a different product_id
recommend_products("e0cf79767c5b016251fe139915c59a26", limit=5)

,id,name,category,price,coPurchaseCount
0,0a4093a4af429dc0a9334300e5c13ae5,Organic Face Mask,beleza_saude,12.99,4
1,0a9a9c25c5a06532d1766e00db833a7f,Vitamin C Serum,beleza_saude,19.99,1
2,ede062bcd13ef64b5cc87369e4edaf62,Essential Oils Collection,beleza_saude,25.00,1
3,aadff88486740e0b0ebe2be6c09476ae,Vitamin C Serum,beleza_saude,19.99,1
4,6abd84909e8ed79ef808c16f90b91093,Vitamin C Serum,beleza_saude,19.99,1


Conclusion: The recommendation system works perfectly and can be used for the store.